In [1]:
import unittest
from run import setup
from interface import handle_query
import os

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
class TestRunIntegration(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # Initialize the components by calling the setup function from run.py
        cls.index, cls.metadata, cls.retriever_model, cls.model, cls.tokenizer, cls.streamer = setup()

        # Define retrieval and generation parameters as used in run.py
        cls.retriever_params = {
            "initial_top_k": 30,
            "final_top_k": 2,
            "similarity_threshold": 1.1
        }
        cls.generator_params = {
            "max_new_tokens": 120,
            "temperature": 0.13,
            "repetition_penalty": 1.2,
            "top_p": 0.96,
            "do_sample": False
        }

    def test_handle_query_with_relevant_context(self):
        """Test the pipeline with a query expected to have relevant context in the metadata."""
        query = "What was discussed about ethics in AI?"

        # Pass the query through handle_query, simulating the chatbot behavior
        response = handle_query(
            query=query,
            index=self.index,
            metadata=self.metadata,
            retriever_model=self.retriever_model,
            model=self.model,
            tokenizer=self.tokenizer,
            streamer=self.streamer,
            retriever_params=self.retriever_params,
            generator_params=self.generator_params
        )

        # Assertions to check that the response is well-formed and relevant
        self.assertIsInstance(response, str, "Response should be a string.")
        self.assertGreater(len(response), 0, "Response should not be empty.")
        self.assertIn("ethics", response.lower(), "Response should mention 'ethics' if relevant context is found.")

    def test_handle_query_with_no_relevant_context(self):
        """Test the pipeline with a query expected to have no relevant context in the metadata."""
        query = "Tell me about a topic not covered in the conference."

        # Pass the query through handle_query, expecting no relevant context
        response = handle_query(
            query=query,
            index=self.index,
            metadata=self.metadata,
            retriever_model=self.retriever_model,
            model=self.model,
            tokenizer=self.tokenizer,
            streamer=self.streamer,
            retriever_params=self.retriever_params,
            generator_params=self.generator_params
        )

        # Assertions to check that the response indicates no relevant context was found
        self.assertIsInstance(response, str, "Response should be a string.")
        self.assertIn("Temat nie wydaje się być poruszany", response, "Expected message when no relevant context is found.")



In [3]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

FAISS index loaded successfully.
Metadata loaded successfully.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Retriever model initialized successfully.


.F
FAIL: test_handle_query_with_relevant_context (__main__.TestRunIntegration)
Test the pipeline with a query expected to have relevant context in the metadata.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-e4254e6f6e7a>", line 42, in test_handle_query_with_relevant_context
    self.assertIn("ethics", response.lower(), "Response should mention 'ethics' if relevant context is found.")
AssertionError: 'ethics' not found in 'temat nie wydaje się być poruszany na tej konferencji.' : Response should mention 'ethics' if relevant context is found.

----------------------------------------------------------------------
Ran 2 tests in 9.333s

FAILED (failures=1)


Query: Tell me about a topic not covered in the conference.
Retrieved in 1731441018.0714 seconds
Average similarity distance (initial top-k): 1.6245
Average similarity distance (final top-k): 1.4727
Results count after filtering: 2

Nie znaleziono odpowiedniego kontekstu: Temat nie był poruszany na konferencji.
Query: What was discussed about ethics in AI?
Retrieved in 1731441018.0852 seconds
Average similarity distance (initial top-k): 1.6854
Average similarity distance (final top-k): 1.5040
Results count after filtering: 2

Nie znaleziono odpowiedniego kontekstu: Temat nie był poruszany na konferencji.
